# Clustering Crypto

In [219]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [220]:
# Load the crypto_data.csv dataset.
file_path = "resources/crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [221]:
crypto_df.dtypes

CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [222]:
crypto_df.count()

CoinName           1252
Algorithm          1252
IsTrading          1252
ProofType          1252
TotalCoinsMined     744
TotalCoinSupply    1252
dtype: int64

In [223]:
# Keep all the cryptocurrencies that are being traded.
crypto_df.loc[(crypto_df["IsTrading"] == "True")]
crypto_df.head()              

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [224]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df['Algorithm'].dropna()


42       Scrypt
365         X11
404      Scrypt
611     SHA-256
808     SHA-256
         ...   
XBC      Scrypt
DVTC     Scrypt
GIOT     Scrypt
OPSC    SHA-256
PUNK        PoS
Name: Algorithm, Length: 1252, dtype: object

In [225]:
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [226]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(["IsTrading"], axis=1)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [227]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()

In [228]:
# Keep the rows where coins are mined.
crypto_df.loc[(crypto_df["TotalCoinsMined"] > 0)]
crypto_df.head() 

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [229]:
# Create a new DataFrame that holds only the cryptocurrencies names.
new_crypto_df = crypto_df.drop(["Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply"], axis=1)
new_crypto_df.head()

,CoinName
42,42 Coin
404,404Coin
808,808
1337,EliteCoin
BTC,Bitcoin


In [230]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(["CoinName"], axis=1)
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
808,SHA-256,PoW/PoS,0.000000e+00,0
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000
XBC,Scrypt,PoS,1.283270e+05,1000000


In [231]:
# Use get_dummies() to create variables for text features.
dum_crypto_df = pd.get_dummies(data = crypto_df, columns=["Algorithm","ProofType"])
dum_crypto_df.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
808,0.000000e+00,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [232]:
# Standardize the data with StandardScaler().
dum_crypto_scaled = StandardScaler().fit_transform(dum_crypto_df)
print(dum_crypto_scaled)

[[-0.10047714 -0.03668886 -0.03668644 ... -0.03668644 -0.03668644
  -0.03668644]
 [-0.07317373 -0.0366887  -0.03668644 ... -0.03668644 -0.03668644
  -0.03668644]
 [-0.10047714 -0.03668886 -0.03668644 ... -0.03668644 -0.03668644
  -0.03668644]
 ...
 [-0.10028834 -0.03668885 -0.03668644 ... -0.03668644 -0.03668644
  -0.03668644]
 [-0.10047382 -0.03668886 -0.03668644 ... -0.03668644 -0.03668644
  -0.03668644]
 [-0.09992104 -0.03668883 -0.03668644 ... -0.03668644 -0.03668644
  -0.03668644]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [233]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

In [250]:
# Create a DataFrame with the three principal components.
crypto_pca = pca.fit_transform(dum_crypto_scaled)
crypto_pca

array([[-0.11862443,  1.22661142, -0.39178498],
       [-0.10416563,  1.22541399, -0.39210138],
       [-0.13018284,  0.81262228, -0.19195618],
       ...,
       [-0.39564278, -1.94183474,  0.12257181],
       [-0.07078396,  0.8009275 , -0.18497029],
       [-0.11832995,  1.22658703, -0.39179142]])

In [270]:
pcs_df = pd.DataFrame(data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"],index=crypto_df.index )
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.118624,1.226611,-0.391785
404,-0.104166,1.225414,-0.392101
808,-0.130183,0.812622,-0.191956
1337,0.381144,1.921248,-0.360868
BTC,-0.284024,-1.394469,0.124647


In [269]:
pcs_df.count

<bound method DataFrame.count of          PC 1      PC 2      PC 3
0   -0.267249  1.289683 -0.382987
1   -0.252167  1.288804 -0.383245
2   -0.160155  0.816076 -0.269652
3    0.288460  1.978167 -0.398429
4   -0.176147 -1.470605  0.070455
..        ...       ...       ...
739 -0.267035  1.289670 -0.382991
740 -0.080633 -2.319435  0.194269
741 -0.110134 -2.157809  0.180661
742 -0.084704  0.872393 -0.109568
743 -0.266941  1.289665 -0.382993

[744 rows x 3 columns]>

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [236]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE


In [237]:
inertia = []
k = list(range(1, 11))

In [238]:
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

In [239]:
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [262]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 0, 0, 2, 2,
       2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2,
       2, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 2, 0, 0,
       0, 0, 0, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 0, 2,
       0, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 0, 0, 0, 2,
       2, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2,
       0, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 2, 2,
       2, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2,
       0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 0, 0,
       0, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2,
       2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 2,
       2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2,
       2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2,

In [271]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.

#clustered_df = crypto_df.join(pcs_df, how='inner')
clustered_df = pd.concat([crypto_df, pcs_df], axis=1)


#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
#clustered_df = clustered_df.join(new_crypto_df, how='inner')
clustered_df["CoinName"] = new_crypto_df["CoinName"]


#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.

#df = pd.DataFrame(predictions, columns=['Class'])
clustered_df["Class"] = model.labels_


# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(744, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.118624,1.226611,-0.391785,42 Coin,2
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.104166,1.225414,-0.392101,404Coin,2
808,SHA-256,PoW/PoS,0.000000e+00,0,-0.130183,0.812622,-0.191956,808,2
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.381144,1.921248,-0.360868,EliteCoin,2
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.284024,-1.394469,0.124647,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.337320,-2.045283,0.213292,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.271847,-0.980531,-0.075195,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.145559,1.383786,-0.291880,Dash,2
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.347377,-2.241509,0.189333,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.337242,-2.045289,0.213291,Ethereum Classic,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [242]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE


In [243]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE

In [244]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

In [245]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE

In [246]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

NameError: name 'plot_df' is not defined

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
